Let's start with a point source propagator of the anisotropic clover action.

In [1]:
import cupy as cp

from pyquda import core, init
from pyquda.utils import io
from pyquda.field import LatticeInfo

Now we imported the `cupy` package. We also imported some modules and functions from `pyquda` package.

PyQUDA is basically a wrapper of QUDA. We need to call `init()` at the first, just like `initQuda()`. `core` holds some import functions, and `io` can read or write some common formats. `LatticeInfo` is the class to describe the lattice.

In [2]:
xi_0, nu = 2.464, 0.95
kappa = 0.115
mass = 1 / (2 * kappa) - 4
coeff_r, coeff_t = 0.91, 1.07

Now the basic parameters are set.

In [3]:
init([1, 1, 1, 1], backend="cupy", resource_path=".cache")
latt_info = LatticeInfo(latt_size=[4, 4, 4, 8], t_boundary=-1, anisotropy=xi_0 / nu)

PyQUDA INFO: Using GPU grid [1, 1, 1, 1]
PyQUDA INFO: Using QUDA_RESOURCE_PATH=.cache
PyQUDA INFO: Using CUDA backend cupy


Disabling GPU-Direct RDMA access
Enabling peer-to-peer copy engine and direct load/store access
QUDA 1.1.0 (git 1.1.0-493876e11-sm_60)
CUDA Driver version = 12020
CUDA Runtime version = 11080
Graphic driver version = 535.161.08
Found device 0: Tesla P100-PCIE-16GB
Found device 1: Tesla P100-PCIE-16GB
Found device 2: Tesla P100-PCIE-16GB
Found device 3: Tesla P100-PCIE-16GB
Using device 0: Tesla P100-PCIE-16GB
cublasCreated successfully


Now we initilized the QUDA with one process and the grid shape is `[1, 1, 1, 1]`. We also get a lattice of size `[4, 4, 4, 8]` with anti-periodic t boundary condition and anisotropy `xi_0 / nu`.

In [4]:
dirac = core.getClover(latt_info, mass, 1e-12, 1000, xi_0, coeff_t, coeff_r)
gauge = io.readQIOGauge("../tests/weak_field.lime")
gauge.stoutSmear(1, 0.125, 4)
io.writeKYUGauge("a.kyu", gauge)

We get a fermion action with corresponing parameters. The action contains a solver with `tol=1e-12, maxiter=1000`. The `gauge` is read from a `.lime` file. The file is generated by Chroma.

In [5]:
dirac.loadGauge(gauge)
propagator = core.invert(dirac, "point", [0, 0, 0, 0])
dirac.destroy()

PyQUDA INFO: Time = 0.491 secs, Performance = 14.407 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.766 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.881 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.972 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.995 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.881 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 14.016 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.862 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.892 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.887 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.986 GFLOPS
PyQUDA INFO: Time = 0.005 secs, Performance = 13.964 GFLOPS


BiCGstab: Convergence at 28 iterations, L2 relative residual: iterated = 3.006119e-13, true = 3.006119e-13 (requested = 1.000000e-12)
BiCGstab: Convergence at 28 iterations, L2 relative residual: iterated = 6.476205e-13, true = 6.476205e-13 (requested = 1.000000e-12)
BiCGstab: Convergence at 28 iterations, L2 relative residual: iterated = 4.205166e-13, true = 4.205166e-13 (requested = 1.000000e-12)
BiCGstab: Convergence at 28 iterations, L2 relative residual: iterated = 4.054432e-13, true = 4.054432e-13 (requested = 1.000000e-12)
BiCGstab: Convergence at 27 iterations, L2 relative residual: iterated = 6.499610e-13, true = 6.499610e-13 (requested = 1.000000e-12)
BiCGstab: Convergence at 28 iterations, L2 relative residual: iterated = 8.191995e-13, true = 8.191995e-13 (requested = 1.000000e-12)
BiCGstab: Convergence at 28 iterations, L2 relative residual: iterated = 5.595534e-13, true = 5.595534e-13 (requested = 1.000000e-12)
BiCGstab: Convergence at 28 iterations, L2 relative residual: 

The `propagator` is solved by QUDA. The source is set to be a point source at `[0, 0, 0, 0]`.

`dirac.destory()` is used to release the multigrid subspace and is not actually necessary in this example.

In [6]:
propagator_chroma = io.readQIOPropagator("../pt_prop_1")
propagator_chroma.toDevice()
print(cp.linalg.norm(propagator.data - propagator_chroma.data))

1.40127023209267e-12


Just like the gauge field, we read a propagator generated by Chroma into PyQUDA. Then we calculate the difference between the result from Chroma and PyQUDA.